해당 repository 및 code들은 wikidocs에서 제공하는 'Pytorch로 시작하는 딥 러닝 입문'을 참고하였음을 밝힙니다. 해당 자료를 바탕으로 숙지한 개념과 추가적인 저의 생각을 기록할 계획입니다. 출처는 다음 사이트와 같습니다.<br><br>
* https://wikidocs.net/book/2788

~~~python
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

model = BinaryClassifier()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

~~~

'클래스로 파이토치 모델 구현하기' 장을 마치면서 짚어봐야 할 점은 파이토치는 클래스 기반으로 짜여진 코드가 많기 때문에 이에 익숙해지고 적용할 능력을 기를 필요성이 있다. 위의 코드가 그 예시이다. 중요하게 봐야 할 곳은 BinaryClassifier를 클래스로 구현한 부분이라고 할 수 있겠다. Keras와 비교해보면 파이토치는 좀 더 세세한 부분을 컨트롤한다는 느낌을 받는다. 특히 optimizer.zero_grad(), cost.backward(), optimizer.step()의 구현은 어떻게 보면 당연히 돌아가야 할 부분이 사용자가 직접 구현하도록 되어 있다. 처음에는 귀찮다고 생각했는데, 추상적으로만 생각했던 로직을 명시적으로 보여주기 때문에 좀 더 구체적으로 이해할 수 있고 많은 부분을 컨트롤 할 수 있다. 